# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# checks the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print("Connection Worked !!")
except Exception as e:
    print(f"Connection Failed, back to coding board !! Error : {e}")

Connection Worked !!


#### Create Keyspace

In [6]:
#Creating a Keyspace query 
keyspaceQuery = """CREATE KEYSPACE IF NOT EXISTS sparkify 
                    with REPLICATION = 
                    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
                """

# Creating Keyspace
try:
    session.execute(keyspaceQuery)
    print("keySpace created !!")
except Exception as e:
    print(f"No keyspace for you, try again!! Error : {e}")

keySpace created !!


#### Set Keyspace

In [7]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
    print("keySpace set !!")
except Exception as e:
    print (f"Keyspace not set, try again!! Error : {e}")

keySpace set !!


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
# Droping table if exists
session.execute("DROP TABLE IF EXISTS sparkify.song_data ;")

# Creating table for query1
create_table1 = """CREATE TABLE IF NOT EXISTS song_data""" 
create_table1 = create_table1 + """ (sessionId INT, itemInSession INT, song_title TEXT, artist TEXT, 
                                    length FLOAT, PRIMARY KEY (sessionId, itemInSession))"""

try: 
    session.execute(create_table1)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")    
    
select_query1 = """SELECT artist, song_title, length FROM song_data
                   WHERE sessionId = %s 
                   AND itemInSession = %s 
                """    

Table Created!!


In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigns the INSERT statements into the `query` variable
        insertQuery = "INSERT INTO song_data(sessionId, itemInSession, song_title, artist, length)"
        insertQuery = insertQuery + " VALUES (%s, %s, %s, %s, %s)"
        ## Assigns which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insertQuery, (int(line[8]), int(line[3]), line[9], line[0], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Added in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(select_query1, (338, 4))
    print("row variable took !!")
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.song_title, "\nLength: "+str(row.length))   

row variable took !!
Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Length: 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
# Droping table if exists
session.execute("DROP TABLE IF EXISTS sparkify.artist_by_user ;")

# Creating table for query2
create_table2 = """CREATE TABLE IF NOT EXISTS artist_by_user""" 
create_table2 = create_table2 +   """ (userID INT, sessionId INT, itemInSession INT,   
                                        artist TEXT, song_title TEXT, users_first_name TEXT, 
                                        users_last_name TEXT, PRIMARY KEY ((userID, sessionId), itemInSession))
                                        WITH CLUSTERING ORDER BY (itemInSession ASC)
                                  """

try: 
    session.execute(create_table2)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")             

Table Created!!


In [12]:
# Insertion for Table 2
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigns the INSERT statements into the `query` variable
        insertQuery = "INSERT INTO artist_by_user(itemInSession, userID, sessionId, artist, song_title, users_first_name, users_last_name)"
        insertQuery = insertQuery + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assigns which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insertQuery, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
## Added for Query 2 in the SELECT statement to verify the data was entered into the table
select_query2 = """SELECT artist, song_title, users_first_name, users_last_name FROM artist_by_user
                   WHERE userID = 10 
                   AND sessionId = 182 
                """           

try:
    rows = session.execute(select_query2)
    print("row variable took !!")
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.song_title, "\nUser's First Name: "+row.users_first_name, "\nUser's Last Name: "+row.users_last_name)

row variable took !!


In [14]:
## Query 3: Gives me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Droping table if exists
session.execute("DROP TABLE IF EXISTS sparkify.user_by_Song ;")

# Creating table for query3
create_table3 = """CREATE TABLE IF NOT EXISTS user_by_Song""" 
create_table3 = create_table3 + """ (song_title TEXT,  userID INT, users_first_name TEXT, users_last_name TEXT,
                                    PRIMARY KEY(song_title, userId))
                                """

try: 
    session.execute(create_table3)
    print("Table Created!!")
except Exception as e:
    print(f"Table creation failed!! Error : {e}")                      

Table Created!!


In [15]:
# Insertion for Table 3
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigns the INSERT statements into the `query` variable
        insertQuery = "INSERT INTO user_by_Song(song_title, userID, users_first_name, users_last_name)"
        insertQuery = insertQuery + " VALUES (%s, %s, %s, %s)"
        ## Assigns which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(insertQuery, (line[9], int(line[10]), line[1], line[4]))

In [16]:
## Added for Query 3 in the SELECT statement to verify the data was entered into the table
select_query3 = """SELECT users_first_name, users_last_name FROM user_by_Song
                   WHERE song_title = %s
                """    

try:
    rows = session.execute(select_query3, ('All Hands Against His Own', ))
    print("row variable took !!")
except Exception as e:
    print(e)
    
for row in rows:
    print("User's First Name: "+row.users_first_name, "\nUser's Last Name: "+row.users_last_name)

row variable took !!
User's First Name: Jacqueline 
User's Last Name: Lynch
User's First Name: Tegan 
User's Last Name: Levine
User's First Name: Sara 
User's Last Name: Johnson


### Drop the tables before closing out the sessions

In [17]:
#Dropping the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS song_data")
session.execute("DROP TABLE IF EXISTS artist_by_user")
session.execute("DROP TABLE IF EXISTS user_by_Song")

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()